In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import timeit
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree
from sklearn import datasets

In [11]:
## Coreset Algorithm
clusters = {}
centroids = []
np_data = []
total_for_coreset = 0
#function to generate 'num_points' random points of 'dim' dimensions.
def generate_data(data, m):
	dim = data.shape[1]
	rows = data.shape[0]
	data_mean = np.mean(data, axis = 0)
	distance = 0
	for point in data:
		distance += np.sum(np.square(point-data_mean))
	prob_dist = []
	#calculating proposal  distribution for each row
	for point in data:
		value=((0.5*(1/n))+0.5*((np.sum(np.square(point-data_mean)))/distance))
		prob_dist.append(value)
	df = pd.DataFrame(prob_dist)
	data = pd.DataFrame(data)
	data['Prob_dist'] = df
	weight_value = []
	for i in range(n):
		weight = 1/(m*prob_dist[i])
		weight_value.append(weight)
	df = pd.DataFrame(weight_value)
	data['weight_value'] = df
	sampled_points = data.sample(n=m, frac=None, replace=False, weights='weight_value', random_state=None, axis=None)
	return sampled_points.iloc[:,:dim]

In [12]:
data = pd.read_csv('mMiniBoone.csv',sep=",")
data = np.array(data)
data = data[:,:data.shape[1]-1]
n = data.shape[0]

In [13]:
# Identifying data length

k = 6
## Finding the number of attributes.
features = data.shape[1]
start = timeit.default_timer()
points = np.array(generate_data(data, k))
stop = timeit.default_timer()
total_for_coreset = stop-start

In [14]:
## Running KMeans on train data.
start = timeit.default_timer()
y_predict = KMeans(n_clusters = k, init = points, n_init =1).fit(data)
stop = timeit.default_timer()
time_for_kmeans = stop-start

In [15]:
# No. of. Nearest Neighbors
n_n = 7

## Storing Cluster centers in cluster_centers
cluster_centers = np.array(y_predict.cluster_centers_)

## generating Query point
query = []
for i in range(0,features):
    x = round(random.random()*random.choice([1,10,-1,-10]),random.choice([1,2,3,4,5]))
    query.append(x)

query = np.array(query)
query = query.reshape(1,features)


## Finding the nearest cluster to query point.
    ## Finding the euclidean distance from first cluster_center point.
start = timeit.default_timer()
dist = 0
for i in range(features):
   dist += (query[0,i]-cluster_centers[0,i])**2
dist = np.sqrt(dist)
near_cluster = 0
i = 1

 ## Finding the euclidean distance from other cluster_center points.
while i < k:
    dist_j = 0
    for j in range(features):
       dist_j += (query[0,j]-cluster_centers[i,j])**2
    dist_j = np.sqrt(dist_j)
    if(dist_j < dist):
        dist = dist_j
        near_cluster = i
    i += 1
stop = timeit.default_timer()   
time_for_closest_center = stop-start   
## Finding the cluster elements to construct KD-Tree

cluster_values = np.ones([n,features])
j = 0
for i in range(n):
    if y_predict.labels_[i] == near_cluster:
        cluster_values[j] = data[i]
        j += 1
cluster_values = cluster_values[:j,:j]

## KDTree Construction and Querying
start = timeit.default_timer()
tree = KDTree(cluster_values)
stop = timeit.default_timer()
time_for_kdtree = stop-start

start = timeit.default_timer()
dist_main , ind_main = tree.query(query[0:1],n_n)
stop = timeit.default_timer()
query_time = stop-start


j=0
for i in ind_main:
    print(cluster_values[i], dist_main[j])
    j += 1
print(total_for_coreset,time_for_kmeans,time_for_closest_center,time_for_kdtree,query_time)
print(total_for_coreset + time_for_kmeans + time_for_closest_center + time_for_kdtree + query_time)

[[ 6.79182e+00  3.52615e+00  2.04477e+01  2.29862e-01  8.36820e-03
   5.01011e-02  4.09843e-01  5.87433e-01  3.69922e+00  8.06710e-02
   4.70206e+00  1.77586e+02  8.88022e-01  7.74065e-01  1.18849e+00
   2.12739e+02  8.58391e-02  1.87207e+00  2.55230e-01  9.36215e-01
  -4.03771e+00  2.46862e-01  3.05886e+01  2.69652e+00  4.61776e-01
  -5.98873e+00  5.69737e+01  1.60055e+00  5.47354e-02 -3.04448e-01
   7.49562e+00  1.63321e+00  9.24188e-01  1.99457e+02  1.36342e-02
   4.21728e-01  3.81695e-02 -1.79200e+00  7.56985e-01  2.11595e-01
   1.11086e+02 -1.55602e+01  1.41019e+00  1.69018e+01  5.43933e-02
   2.07662e-01 -1.48845e+00  3.04907e+00  2.52467e+00  2.92300e-01]
 [ 6.20906e+00  4.58753e+00  1.97665e+01  2.53888e-01  8.69565e-03
   2.18434e-01  6.10289e-01  6.94149e-01  3.70923e+00  1.69628e-01
   4.77909e+00  1.65662e+02  9.17132e-01  7.30796e-01  1.10619e+00
   1.84733e+02  7.70006e-02  1.45662e+00  2.73913e-01  2.09208e+00
  -3.25642e+00  2.95652e-01  2.07142e+01  2.80337e+00  4.1090

In [20]:
data.shape

(130062, 50)

In [16]:
start = timeit.default_timer()
tree = KDTree(data)
stop = timeit.default_timer()
total_time = stop-start
print(total_time)
j=0
start = timeit.default_timer()
dist_dum , ind_dum = tree.query(query,n_n)
stop = timeit.default_timer()
total_time = stop-start
print(total_time)
for i in ind_dum:
    print(data[i],dist_dum[j])
    j =+ 1

0.7654355359991314
0.0017975299997488037
[[ 6.79182e+00  3.52615e+00  2.04477e+01  2.29862e-01  8.36820e-03
   5.01011e-02  4.09843e-01  5.87433e-01  3.69922e+00  8.06710e-02
   4.70206e+00  1.77586e+02  8.88022e-01  7.74065e-01  1.18849e+00
   2.12739e+02  8.58391e-02  1.87207e+00  2.55230e-01  9.36215e-01
  -4.03771e+00  2.46862e-01  3.05886e+01  2.69652e+00  4.61776e-01
  -5.98873e+00  5.69737e+01  1.60055e+00  5.47354e-02 -3.04448e-01
   7.49562e+00  1.63321e+00  9.24188e-01  1.99457e+02  1.36342e-02
   4.21728e-01  3.81695e-02 -1.79200e+00  7.56985e-01  2.11595e-01
   1.11086e+02 -1.55602e+01  1.41019e+00  1.69018e+01  5.43933e-02
   2.07662e-01 -1.48845e+00  3.04907e+00  2.52467e+00  2.92300e-01]
 [ 6.20906e+00  4.58753e+00  1.97665e+01  2.53888e-01  8.69565e-03
   2.18434e-01  6.10289e-01  6.94149e-01  3.70923e+00  1.69628e-01
   4.77909e+00  1.65662e+02  9.17132e-01  7.30796e-01  1.10619e+00
   1.84733e+02  7.70006e-02  1.45662e+00  2.73913e-01  2.09208e+00
  -3.25642e+00  2.95

In [19]:
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree
import random
import timeit

# Loading data
data = pd.read_csv('mMiniBoone.csv',sep=",")
data = np.array(data)
data = data[:,:data.shape[1]-1]
# Identifying data length
n = data.shape[0]
k = 6

# No. of. Nearest Neighbors
n_n = 7

## Finding the number of attributes.
features = data.shape[1]


## Running KMeans on train data.
start = timeit.default_timer()
y_predict = KMeans(n_clusters = k).fit(data)
stop = timeit.default_timer()
time_for_kmeans = stop-start
## Storing Cluster centers in cluster_centers
cluster_centers = np.array(y_predict.cluster_centers_)

## Finding the nearest cluster to query point.
    ## Finding the euclidean distance from first cluster_center point.
start = timeit.default_timer()
dist = 0
for i in range(features):
   dist += (query[0,i]-cluster_centers[0,i])**2
dist = np.sqrt(dist)
near_cluster = 0
i = 1

 ## Finding the euclidean distance from other cluster_center points.
while i < k:
    dist_j = 0
    for j in range(features):
       dist_j += (query[0,j]-cluster_centers[i,j])**2
    dist_j = np.sqrt(dist_j)
    if(dist_j < dist):
        dist = dist_j
        near_cluster = i
    i += 1
    
stop = timeit.default_timer()   
time_for_closest_center = stop-start
## Finding the cluster elements to construct KD-Tree
cluster_values = np.ones([n,features])
j = 0
for i in range(n):
    if y_predict.labels_[i] == near_cluster:
        cluster_values[j] = data[i]
        j += 1
cluster_values = cluster_values[:j,:j]

## KDTree Construction and Querying
start = timeit.default_timer()
tree = KDTree(cluster_values, np.log(cluster_values.shape[0]))
stop = timeit.default_timer()
time_for_kdtree = stop-start

start = timeit.default_timer()
dist_main , ind_main = tree.query(query[0:1],n_n)
stop = timeit.default_timer()
query_time = stop-start


j=0
for i in ind_main:
    print(cluster_values[i], dist_main[j])
    j += 1
print(time_for_kmeans,time_for_closest_center,time_for_kdtree,query_time)
print(time_for_closest_center + time_for_kdtree + query_time)

[[ 6.79182e+00  3.52615e+00  2.04477e+01  2.29862e-01  8.36820e-03
   5.01011e-02  4.09843e-01  5.87433e-01  3.69922e+00  8.06710e-02
   4.70206e+00  1.77586e+02  8.88022e-01  7.74065e-01  1.18849e+00
   2.12739e+02  8.58391e-02  1.87207e+00  2.55230e-01  9.36215e-01
  -4.03771e+00  2.46862e-01  3.05886e+01  2.69652e+00  4.61776e-01
  -5.98873e+00  5.69737e+01  1.60055e+00  5.47354e-02 -3.04448e-01
   7.49562e+00  1.63321e+00  9.24188e-01  1.99457e+02  1.36342e-02
   4.21728e-01  3.81695e-02 -1.79200e+00  7.56985e-01  2.11595e-01
   1.11086e+02 -1.55602e+01  1.41019e+00  1.69018e+01  5.43933e-02
   2.07662e-01 -1.48845e+00  3.04907e+00  2.52467e+00  2.92300e-01]
 [ 6.20906e+00  4.58753e+00  1.97665e+01  2.53888e-01  8.69565e-03
   2.18434e-01  6.10289e-01  6.94149e-01  3.70923e+00  1.69628e-01
   4.77909e+00  1.65662e+02  9.17132e-01  7.30796e-01  1.10619e+00
   1.84733e+02  7.70006e-02  1.45662e+00  2.73913e-01  2.09208e+00
  -3.25642e+00  2.95652e-01  2.07142e+01  2.80337e+00  4.1090

In [22]:
cluster_values.shape

(85649, 50)